In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\Ana's PC\Documents\Projects\pfdelta\data_generation`


In [2]:
import PowerModels
import PowerModels: ref, var, ids
const PM = PowerModels
import JuMP
import Ipopt
include("src/build_opf_pfdelta.jl")

variable_branch_power_pfdelta (generic function with 1 method)

In [3]:
net = PM.parse_matpower("pglib/pglib_opf_case5_pjm.m")

[info | PowerModels]: extending matpower format with data: areas 1x3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]


Dict{String, Any} with 14 entries:
  "bus"            => Dict{String, Any}("4"=>Dict{String, Any}("zone"=>1, "bus_…
  "source_type"    => "matpower"
  "name"           => "pglib_opf_case5_pjm"
  "dcline"         => Dict{String, Any}()
  "source_version" => "2"
  "gen"            => Dict{String, Any}("4"=>Dict{String, Any}("vg"=>1.0, "mbas…
  "branch"         => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>0.00108,…
  "storage"        => Dict{String, Any}()
  "switch"         => Dict{String, Any}()
  "baseMVA"        => 100.0
  "per_unit"       => true
  "areas"          => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>Any…
  "shunt"          => Dict{String, Any}()
  "load"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>Any…

In [4]:
pm = PM.instantiate_model(net, PM.ACPPowerModel, build_opf_power_flow_delta)
print(pm.model)

Min 1000 0_pg[5] + 4000 0_pg[4] + 1500 0_pg[2] + 3000 0_pg[3] + 1400 0_pg[1]
Subject to
 0_va[4] == 0.0
 -0_pg[5] + 0_p[(6, 5, 4)] + 0_p[(3, 5, 1)] == 0.0
 -0_qg[5] + 0_q[(6, 5, 4)] + 0_q[(3, 5, 1)] == 0.0
 -0_pg[4] + 0_p[(6, 4, 5)] + 0_p[(5, 4, 3)] + 0_p[(2, 4, 1)] == -4.0
 -0_qg[4] + 0_q[(6, 4, 5)] + 0_q[(5, 4, 3)] + 0_q[(2, 4, 1)] == -1.3147
 0_p[(4, 2, 3)] + 0_p[(1, 2, 1)] == -3.0
 0_q[(4, 2, 3)] + 0_q[(1, 2, 1)] == -0.9861
 -0_pg[3] + 0_p[(5, 3, 4)] + 0_p[(4, 3, 2)] == -3.0
 -0_qg[3] + 0_q[(5, 3, 4)] + 0_q[(4, 3, 2)] == -0.9861
 -0_pg[2] - 0_pg[1] + 0_p[(2, 1, 4)] + 0_p[(3, 1, 5)] + 0_p[(1, 1, 2)] == 0.0
 -0_qg[2] - 0_qg[1] + 0_q[(2, 1, 4)] + 0_q[(3, 1, 5)] + 0_q[(1, 1, 2)] == 0.0
 0_vm[5] >= 0.9
 0_vm[4] >= 0.9
 0_vm[3] >= 0.9
 0_vm[1] >= 0.9
 0_pg[5] >= 6.0
 0_pg[2] >= 1.7
 0_pg[3] >= 5.2
 0_pg[1] >= 0.4
 0_vm[5] <= 1.1
 0_vm[4] <= 1.1
 0_vm[3] <= 1.1
 0_vm[1] <= 1.1
 0_p[(5, 3, 4)] - (((3.3336667000033335 + 0.0) / 1.0 ^ 2.0) * 0_vm[3] ^ 2.0 + ((-3.3336667000033335 * 1.0 + -33.3

In [5]:
result = PM.optimize_model!(pm, optimizer=Ipopt.Optimizer)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      155
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      240

Total number of variables............................:       44
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equal

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.848
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => -2.18633e5
  "solution"           => Dict{String, Any}("baseMVA"=>100.0, "gen"=>Dict{Strin…
  "objective_lb"       => -Inf

In [10]:
result["solution"]["gen"]

Dict{String, Any} with 5 entries:
  "4" => Dict{String, Any}("qg"=>90.4866, "pg"=>-86.6621)
  "1" => Dict{String, Any}("qg"=>27.0972, "pg"=>0.4)
  "5" => Dict{String, Any}("qg"=>44.9625, "pg"=>109.305)
  "2" => Dict{String, Any}("qg"=>23.9682, "pg"=>1.7)
  "3" => Dict{String, Any}("qg"=>16.1102, "pg"=>5.2)